In [1]:
!pip uninstall tensorflow -y
!pip install tensorflow==2.12.0

Found existing installation: tensorflow 2.12.0
Uninstalling tensorflow-2.12.0:
  Successfully uninstalled tensorflow-2.12.0
  Using cached tensorflow-2.12.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (585.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.12.0 which is incompatible.


# 스태킹 앙상블

In [2]:
 # 기본 라이브러리를 불러오기
import pandas as pd
import numpy as np
# 시각화 라이브러리 불러오기
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# 변환을 위한 라이브러리 불러오기
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
# 통계 모델 라이브러리 불러오기
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
import xgboost as xgb
# 딥러닝 라이브러리도 일단 불러오기
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# 성과지표를 계산하기 위한 라이브러리 불러오기
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import warnings
warnings.filterwarnings("ignore")

In [3]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()

In [4]:
X_data=data.data
y_data=data.target
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=1999)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2)
pd.DataFrame(X_train).head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,11.99,24.89,77.61,441.3,0.10300,0.09218,0.05441,0.04274,0.1820,0.06850,...,12.98,30.36,84.48,513.9,0.13110,0.1822,0.16090,0.12020,0.2599,0.08251
1,15.30,25.27,102.40,732.4,0.10820,0.16970,0.16830,0.08751,0.1926,0.06540,...,20.27,36.71,149.30,1269.0,0.16410,0.6110,0.63350,0.20240,0.4027,0.09876
2,20.64,17.35,134.80,1335.0,0.09446,0.10760,0.15270,0.08941,0.1571,0.05478,...,25.37,23.17,166.80,1946.0,0.15620,0.3055,0.41590,0.21120,0.2689,0.07055
3,11.14,14.07,71.24,384.6,0.07274,0.06064,0.04505,0.01471,0.1690,0.06083,...,12.12,15.82,79.62,453.5,0.08864,0.1256,0.12010,0.03922,0.2576,0.07018
4,10.26,14.71,66.20,321.6,0.09882,0.09159,0.03581,0.02037,0.1633,0.07005,...,10.88,19.48,70.89,357.1,0.13600,0.1636,0.07162,0.04074,0.2434,0.08488


In [6]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(455, 30) (114, 30)
(455,) (114,)


In [7]:
# 개별 모델
svm=SVC(random_state=0)
rf=RandomForestClassifier(n_estimators=100, random_state=0)
lr=LogisticRegression()

In [8]:
from lightgbm import LGBMClassifier

In [19]:
#최종모델
lgbm=LGBMClassifier(verbose=-1)

In [20]:
svm.fit(X_train, y_train)
rf.fit(X_train, y_train)
lr.fit(X_train, y_train)

LogisticRegression()

In [21]:
svm_pred=svm.predict(X_test)
rf_pred=rf.predict(X_test)
lr_pred=lr.predict(X_test)
print("svm:{0:.4f}, rf:{1:.4f}, lr:{2:.4f}".format(accuracy_score(y_test, svm_pred),
                                                   accuracy_score(y_test, rf_pred),
                                                   accuracy_score(y_test, lr_pred)))

svm:0.9211, rf:0.9649, lr:0.9649


In [22]:
new_data=np.array([svm_pred, rf_pred, lr_pred])
new_data.shape

(3, 114)

In [23]:
new_data=np.transpose(new_data)
new_data.shape

(114, 3)

In [24]:
new_data[:10]

array([[0, 0, 0],
       [1, 1, 1],
       [0, 0, 0],
       [1, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]])

In [25]:
lgbm.fit(new_data, y_test)
lgbm_pred=lgbm.predict(new_data)
print("정확도: {0:.4f}".format(accuracy_score(y_test, lgbm_pred)))

정확도: 0.9825
